In [1]:
from bs4 import BeautifulSoup
import requests

In [2]:
website_url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

soup = BeautifulSoup(website_url,"lxml")
print(soup.prettify())

In [4]:
My_table = soup.find("table",{"class":"wikitable sortable"})

My_table

In [6]:
#creating lists for each of the columns I know to be in my table.
A=[]
B=[]
C=[]

In [7]:
#utilizing HTML tags for rows <tr> and elements <td> to iterate through each row of data and append data elements to their appropriate lists:
for row in My_table.findAll("tr"):
    cells = row.findAll('td')
    if len(cells)==3: #Only extract table body not heading
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

In [8]:
#import pandas to convert list to data frame
import pandas as pd
df=pd.DataFrame(A,columns=['Postcode'])
df['Borough']=B
df['Neighbourhood']=C

In [9]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [10]:
#To drop the rows which has "not assigned" in them, I change the data type into string 
df.dtypes
df = df.astype(str)
df_new = df[~df.Borough.str.contains("assigned")]

df_new

In [12]:
#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough:
import numpy as np
df_new['Neighbourhood'] = np.where(df_new['Neighbourhood'] == "Not assigned\n", df_new['Borough'], df_new['Neighbourhood'])

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


df_new

In [14]:
# More than one neighborhood can exist in one postal code area:

df_new = df_new.groupby(['Postcode','Borough'])['Neighbourhood'].apply(','.join).reset_index()

df_new

In [16]:
#  number of rows of your dataframe.
df_new.shape[0]

103